This notebook queries the Wikidata API for relational knowledge, saving it in a JSON format that can be read by `dataclasses_json` as Relation objects. The focus is on symmetric relations such as `married` or `is-sibling`, but low-context relations are another area of interest that can be explored further below.

In [2]:
import sys
sys.path.append('..')
from lre.data import Relation, RelationSample
import json
import requests
import pandas as pd
!ls

A2Rel.ipynb        RelBenchmark.py    example.json       text
RelBenchmark.ipynb Wikidata2Rel.ipynb json               wikidata


In [4]:
# Define the endpoint and query
url = "https://query.wikidata.org/sparql"
query = """
SELECT ?person1 ?person1Label ?person2 ?person2Label ?marriageDate ?endDate WHERE {
  ?person1 wdt:P26 ?person2 .
  OPTIONAL { ?person1 wdt:P580 ?marriageDate. }
  OPTIONAL { ?person1 wdt:P582 ?endDate. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
"""

# Send the request to the Wikidata SPARQL endpoint
response = requests.get(url, params={'query': query, 'format': 'json'})
data = response.json()

In [5]:
data

{'head': {'vars': ['person1',
   'person1Label',
   'person2',
   'person2Label',
   'marriageDate',
   'endDate']},
 'results': {'bindings': [{'person1': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q2259'},
    'person2': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q1407'},
    'person1Label': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Julia the Elder'},
    'person2Label': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Tiberius'}},
   {'person1': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q57212'},
    'person2': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q1339'},
    'person1Label': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Anna Magdalena Bach'},
    'person2Label': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Johann Sebastian Bach'}},
   {'person1': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q57487'},
    'person2': {'type': 'uri',
    

In [7]:
relation_name = "person - spouse"
relation_path_json = "json/sym/person-spouse" + ".json"
prompts = [
    "{} is married to",
    ]
# Extract results
RelationSamples = []
for item in data['results']['bindings']:
    spouse = item['person1Label']['value']
    person = item['person2Label']['value']
    RelationSamples.append(RelationSample(person, spouse))

rel = Relation(
                name=relation_name,
                prompt_templates=prompts,
                prompt_templates_zs=prompts,
                samples=
                RelationSamples
            )

relation_json = rel.to_dict()

In [8]:
with open(relation_path_json, "w") as file:
    json.dump(relation_json, file, indent=4)